In [2]:
import os
import cv2
import shutil
import deeplabcut
import numpy as np
import pandas as pd
import subprocess
from datetime import date
import math
from decimal import Decimal

2023-04-28 17:50:20.703722: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 17:50:20.808515: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-28 17:50:21.182416: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/yramakrishna/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:
2023-04-28 17:50:21.182469: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic l

Loading DLC 2.3.3...


In [3]:
os.getcwd()

'/home/yramakrishna/DeepLabCut/conda-environments/Codes'

In [4]:
if not os.path.exists('All_vids'):
        os.makedirs('All_vids')
os.chdir('All_vids')

In [5]:
#enter path of folder (including folder name) containing all your videos of interest
vid_dir = '/home/yramakrishna/DeepLabCut/conda-environments/Codes/Videos'
to_dir = os.path.join(os.getcwd(), os.path.basename(vid_dir))


In [ ]:
shutil.move(vid_dir, to_dir)

In [ ]:
os.path.dirname(to_dir)

In [ ]:
#Soring converted videos in a new folder
if not os.path.exists('Conv_vids'):
        os.makedirs('Conv_vids')

In [ ]:
input_file = '/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/Videos/ken_20210503_0911_XBI19_reversal_learning_foraging_SC3_DCS_frontcamera.mov'
output_file = os.path.splitext(os.path.basename(input_file))[0].lstrip('.') 
output_file
os.path.join(os.path.dirname(os.path.dirname(input_file)), '/Conv_vids', output_file +'_conv.mp4')

In [ ]:
#Converting .mov videos to .mp4
def conv_mp4(input_file):
    out_name = os.path.splitext(input_file)[0].lstrip('.') 
    print(out_name)
    output_file = out_name + '.mp4'

    # Run FFmpeg command to convert MOV to MP4
    command = f"ffmpeg -i {input_file} -c:v libx264 -preset medium -crf 23 -c:a aac -b:a 128k {output_file}"
    subprocess.call(command, shell=True)
    new = os.path.join(os.path.dirname(os.path.dirname(output_file)), 'Conv_vids', os.path.basename(output_file))
    shutil.move(output_file, new)


In [ ]:
for file in os.listdir(os.path.join(os.path.dirname(to_dir), 'Videos')):
    if file.endswith(".mov"):
        # print(file)
        path=os.path.join(os.path.join(os.path.dirname(to_dir), 'Videos', file))
        conv_mp4(path)
        # print(path)

In [ ]:
# if not os.path.exists('NDSConv_vids'):
#         os.makedirs('NDSConv_vids')

In [22]:
os.chdir(os.path.dirname(to_dir))

In [ ]:
#os.listdir(os.path.join(os.path.dirname(to_dir), 'DSConv_vids'))
# os.path.join(os.path.dirname(to_dir), 'NDSConv_vids', os.path.basename(dsv)) 

In [6]:
if not os.path.exists('Downsize_Conv_vids'):
        os.makedirs('Downsize_Conv_vids')
os.chdir('Downsize_Conv_vids')

In [12]:
os.getcwd()

'/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/Downsize_Conv_vids'

In [8]:
def downsize(path):
    deeplabcut.DownSampleVideo(path, width=320, height=240)

In [8]:
# vid_names = []
for file in os.listdir(os.path.join(os.path.dirname(to_dir), 'Conv_vids')):
    if file.endswith(".mp4"):
        path=os.path.join(os.path.join(os.path.dirname(to_dir), 'Conv_vids', file))
        downsize(path)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [13]:
os.chdir('..')
if not os.path.exists('GC_Conv_vids'):
        os.makedirs('GC_Conv_vids')
os.chdir('GC_Conv_vids')

In [14]:
os.getcwd()

'/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/GC_Conv_vids'

In [15]:
#cropping video and Gamma correction
def gamma_correct(convid):
    
    # Load the video file
    video = cv2.VideoCapture(convid)

    # Get the frame rate and total number of frames
    fps = int(video.get(cv2.CAP_PROP_FPS))
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    #*********************************************

    target_br = 0.6
    target_g = math.log(target_br)

    #**********************************************

    # Set the video writer
    output_name = os.path.splitext(os.path.basename(convid))[0].lstrip('.') + '_GC.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_name, fourcc, fps, (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))))
    start_frame = 0
    # Set the current frame number to the start frame
    video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    frame_number = 0
    # Loop through the frames and write them to the output video
    for i in range(start_frame, total_frames):
        ret, frame = video.read()
        if not ret:
            break
        
        frame_normalized = frame.astype(np.float32) / 255.0
        gray = cv2.cvtColor(frame_normalized, cv2.COLOR_BGR2GRAY)
        
        brightness = np.mean(frame_normalized)

        #finding and correcting gamma - a number is assigned to be the 'innate' gamma value of the frame, and that value is used to find the correction needed. 
        if brightness == 0:
            print('err')
            continue
        assigned_g = math.log(brightness)
        gamma = assigned_g/target_g
        
        # Normalize the pixel values to the range [0, 1]
        # frame_normalized = frame.astype(np.float32) / 255.0

        # Apply gamma correction
        frame_corrected = np.power(frame_normalized, 1/gamma)

        # Scale the pixel values back to the range [0, 255]
        frame_scaled = (frame_corrected * 255.0).astype(np.uint8)
        
        print(round(frame_number/total_frames*100, 2), end = '\r')
        frame_number += 1
        cv2.imshow('output', frame)
        # cv2.imshow('output', frame_scaled)
        out.write(frame_scaled)
    # new = os.path.join(os.path.dirname(os.path.dirname(convid)), 'GC_Conv_vids', os.path.basename(convid))
    # shutil.move(dsv, new)
    print("Done - ", convid)
    print('')
    # Release the video objects
    video.release()
    out.release()
    cv2.destroyAllWindows()

In [31]:

for file in os.listdir(os.path.join(os.path.dirname(to_dir), 'Downsize_Conv_vids')):
    if file.endswith(".mp4"):
        path=os.path.join(os.path.join(os.path.dirname(to_dir), 'Downsize_Conv_vids', file))
        gamma_correct(path)

Done -  /home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/Downsize_Conv_vids/ken_20210505_0939_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled.mp4

Done -  /home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/Downsize_Conv_vids/ken_20210504_1236_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled.mp4

Done -  /home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/Downsize_Conv_vids/vin_20210528_1002_XBI19_reversal_learning_foraging_SC3_DCS_frontCameradownsampled.mp4

Done -  /home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/Downsize_Conv_vids/vin_20210527_1010_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled.mp4

Done -  /home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/Downsize_Conv_vids/vin_20210526_1022_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled.mp4

Done -  /home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/Downsize_Conv_vids/ken_20210506_XBI19_

In [16]:
# gamma_correct('/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/Downsize_Conv_vids/ken_20210505_0939_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled.mp4')
# # gamma_correct('/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/Downsize_Conv_vids/ken_20210503_0911_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled.mp4')

QObject::moveToThread: Current thread (0xc79f460) is not the object's thread (0xc1b7ca0).
Cannot move to target thread (0xc79f460)

QObject::moveToThread: Current thread (0xc79f460) is not the object's thread (0xc1b7ca0).
Cannot move to target thread (0xc79f460)

QObject::moveToThread: Current thread (0xc79f460) is not the object's thread (0xc1b7ca0).
Cannot move to target thread (0xc79f460)

QObject::moveToThread: Current thread (0xc79f460) is not the object's thread (0xc1b7ca0).
Cannot move to target thread (0xc79f460)

QObject::moveToThread: Current thread (0xc79f460) is not the object's thread (0xc1b7ca0).
Cannot move to target thread (0xc79f460)

QObject::moveToThread: Current thread (0xc79f460) is not the object's thread (0xc1b7ca0).
Cannot move to target thread (0xc79f460)

QObject::moveToThread: Current thread (0xc79f460) is not the object's thread (0xc1b7ca0).
Cannot move to target thread (0xc79f460)

QObject::moveToThread: Current thread (0xc79f460) is not the object's thread

KeyboardInterrupt: 

In [ ]:
# os.path.join(os.path.dirname(os.path.dirname(input_file)), 'All_vids/Conv_vids', out_name +'_conv.mp4')
# out_name = os.path.splitext(os.path.basename(path))[0].lstrip('.') + '.mp4'
# out_name

In [36]:
os.chdir('..')


'/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids'

In [6]:
os.getcwd()

'/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids'

In [7]:
folder = [] 
for file in os.listdir(os.path.join(os.path.dirname(to_dir), 'GC_Conv_vids')):
    if file.endswith(".mp4"):
        path=os.path.join(os.path.join(os.path.dirname(to_dir), 'GC_Conv_vids', file))
        folder.append(path)
print(folder)
        
# folder2 = [] 
# for file in os.listdir(os.path.join(os.path.dirname(to_dir), 'Conv_vids')):
#     if file.endswith(".mp4"):
#         path=os.path.join(os.path.join(os.path.dirname(to_dir), 'Conv_vids', file))
#         folder2.append(path)
        
# print(folder, folder2)

['/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/GC_Conv_vids/ken_20210506_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled_GC.mp4', '/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/GC_Conv_vids/nil_20210614_XBI19downsampled_GC.mp4', '/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/GC_Conv_vids/vin_20210604_0907_XBI19_reversal_learning_foraging_SC3_DCS_front_cameradownsampled_GC.mp4', '/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/GC_Conv_vids/ken_20210504_1236_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled_GC.mp4', '/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/GC_Conv_vids/ken_20210505_0939_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled_GC.mp4', '/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/GC_Conv_vids/vin_20210525_0844_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled_GC.mp4', '/home/yramakrishna/DeepLabCut/conda-envi

In [8]:
project_name = 'DLC'
your_name = 'VK'

In [16]:
    bodyparts = ['RightEye_Outer', 'RightEye_Top', 'RightEye_Bottom', 'RightEye_Inner', 'RightEye_Pupil', 'LeftEye_Outer', 'LeftEye_Top', 'LeftEye_Bottom', 'LeftEye_Inner', 'LeftEye_Pupil',
                 'OutlineTop_Mid','RightNostrils_Top', 'RightNostrils_Bottom','LeftNostrils_Top', 'LeftNostrils_Bottom']
    videotype = os.path.splitext(file)[-1].lstrip('.')  # or MOV, or avi, whatever you uploaded!
    video_down = folder
    name_fold = 'Analyse_' + os.path.basename(os.path.dirname(file))
    
    model_options = deeplabcut.create_project.modelzoo.Modeloptions
    model_selection = 'primate_face'

    config_path, train_config_path = deeplabcut.create_pretrained_project(
        project_name,
        your_name,
        video_down,
        videotype=videotype,
        model=model_selection,
        analyzevideo=True,
        createlabeledvideo=False,
        copy_videos=False,
    )

Project "/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/DLC-VK-2023-04-28" already exists!
Loading.... primate_face


/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/DLC-VK-2023-04-28/dlc-models/iteration-0/DLCApr28-trainset95shuffle1/train/pose_cfg.yaml
Analyzing video...
Using snapshot-1030000 for model /home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/DLC-VK-2023-04-28/dlc-models/iteration-0/DLCApr28-trainset95shuffle1


/home/yramakrishna/anaconda3/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
2023-04-28 17:10:18.302623: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-28 17:10:18.302799: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-28 17:10:18.302925: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-28 17:10:18.303097: I tensorflow/stream_exec

Analyzing all the videos in the directory...
Starting to analyze %  /home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/DLC-VK-2023-04-28/videos/ken_20210505_0939_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled_GC.mp4
Loading  /home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/DLC-VK-2023-04-28/videos/ken_20210505_0939_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled_GC.mp4
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


In [13]:
def process(fold, project_name, your_name):
    file = fold[0]
#     print(file)
#     bodyparts = ['RightEye_Outer', 'RightEye_Top', 'RightEye_Bottom', 'RightEye_Inner', 'RightEye_Pupil', 'LeftEye_Outer', 'LeftEye_Top', 'LeftEye_Bottom', 'LeftEye_Inner', 'LeftEye_Pupil',
#                  'OutlineTop_Mid','RightNostrils_Top', 'RightNostrils_Bottom','LeftNostrils_Top', 'LeftNostrils_Bottom']
#     videotype = os.path.splitext(file)[-1].lstrip('.')  # or MOV, or avi, whatever you uploaded!
#     video_down = folder
    name_fold = 'Analyse_' + os.path.basename(os.path.dirname(file))
    
#     model_options = deeplabcut.create_project.modelzoo.Modeloptions
#     model_selection = 'primate_face'

#     config_path, train_config_path = deeplabcut.create_pretrained_project(
#         project_name,
#         your_name,
#         video_down,
#         videotype=videotype,
#         model=model_selection,
#         analyzevideo=True,
#         createlabeledvideo=False,
#         copy_videos=False,
#     )

    edits = {
        'dotsize': 1.5,  # size of the dots!
        'pcutoff': 0.4,  # the higher, the more conservative the plotting!
    }
    deeplabcut.auxiliaryfunctions.edit_config(config_path, edits)
    project_path = os.path.dirname(config_path)
    full_video_path = []
    for i in fold:
        full_video_path.append(os.path.join(project_path,'videos', os.path.basename(i)))

    # filter predictions (should already be done above ;):
    deeplabcut.filterpredictions(config_path, full_video_path, videotype=videotype)

    # re-create the video with your edits!
    # deeplabcut.CropVideo(config_path, full_video_path, 'crop', 
    deeplabcut.create_labeled_video(config_path, full_video_path, videotype=videotype, displayedbodyparts=bodyparts, draw_skeleton = True, filtered=True)
    
    

In [14]:
process(folder, project_name, your_name)

# all_fold = [folder2, folder]
# print(all_fold)

Filtering with median model /home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/DLC-VK-2023-04-28/videos/ken_20210506_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled_GC.mp4
Saving filtered csv poses!
Filtering with median model /home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/DLC-VK-2023-04-28/videos/nil_20210614_XBI19downsampled_GC.mp4
Saving filtered csv poses!
Filtering with median model /home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/DLC-VK-2023-04-28/videos/vin_20210604_0907_XBI19_reversal_learning_foraging_SC3_DCS_front_cameradownsampled_GC.mp4
Saving filtered csv poses!
Filtering with median model /home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/DLC-VK-2023-04-28/videos/ken_20210504_1236_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled_GC.mp4
Saving filtered csv poses!
Filtering with median model /home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/DLC-VK-2023-04-28/videos/ken_202

  0%|                                                 | 0/41316 [00:00<?, ?it/s]

Duration of video [s]: 7250.33, recorded with 9.0 fps!Duration of video [s]: 5035.56, recorded with 16.0 fps!

Overall # of frames: 80569 with cropped frame dimensions: 320 240Overall # of frames: 65253 with cropped frame dimensions: 320 240

Generating frames and creating video.Generating frames and creating video.



  0%|                                       | 76/41316 [00:00<00:55, 749.58it/s]

Duration of video [s]: 8507.27, recorded with 15.0 fps!
Overall # of frames: 127609 with cropped frame dimensions: 320 240
Generating frames and creating video.


...

Duration of video [s]: 17931.22, recorded with 18.0 fps!
Overall # of frames: 322762 with cropped frame dimensions: 320 240
Generating frames and creating video.


  0%|▏                                   | 660/171687 [00:00<00:51, 3352.76it/s]

Duration of video [s]: 16128.03, recorded with 30.0 fps!
Overall # of frames: 483841 with cropped frame dimensions: 320 240
Generating frames and creating video.

  1%|▏                                    | 537/80569 [00:00<00:56, 1416.42it/s]

100%|█████████████████████████████████| 483841/483841 [02:47<00:00, 2895.17it/s]


In [ ]:
# def main():
#     Parallel(n_jobs=1)(delayed(process)(fold, project_name, your_name) for fold in all_fold)


In [ ]:
# main()

In [ ]:

# for fold in all_fold:
#     name_fold = 'Analyse_' + os.path.basename(os.path.dirname(fold[0]))
#     print(name_fold)
#     print(fold)
#     if not os.path.exists(name_fold):
#         os.makedirs(name_fold)
#     os.chdir(name_fold)
#     print(os.getcwd())
#     process(fold, project_name, your_name)
#     os.chdir('..')
#     print(os.getcwd())
    

In [ ]:

# proj_fold

In [10]:
os.chdir('/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids')
h5files = []
vid_to_crop = []
today = str(date.today())
# today = "2023-04-12"
proj_fold = project_name+'-'+your_name+'-'+today
# print(today)
target = os.path.join(os.getcwd(),proj_fold,'videos')
print(target)
for file in os.listdir(target):
    if file.endswith("filtered.h5"):
        h5files.append(os.path.join(target,file))
for file in os.listdir(target):   
    if file.endswith("labeled.mp4"):
        vid_to_crop.append(os.path.join(target,file))
        
# print(h5files, vid_to_crop)
# videotype = os.path.splitext(file)[-1].lstrip('.')

/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids/DLC-VK-2023-04-28/videos


In [11]:
def crop_it(feed):
    
    fl_pth, vid_pth, x = feed
    # deeplabcut.analyze_videos(config_path, 'CropGr.mp4', save_as_csv=True, dynamic=(True,.6,30))
    print("Starting Cropping")

    # Load tracking results generated by DeepLabCut
    tracking_data = pd.read_hdf(fl_pth)

    # Define the names of the facial features that you want to extract frames for
    feature_names = ['RightEye_Pupil','LeftEye_Pupil', 'NostrilsTop_Centre', 'OutlineTop_Mid']

    # Define the threshold for the confidence score of the facial features
    confidence_threshold = 0.9
    
    # Load the input video
    cap = cv2.VideoCapture(vid_pth)
    
    tot_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Define the output video writer
    bn = os.path.basename(vid_pth).split('.')[0]
    out_file = 'AAACrpoutput_video' + bn + '.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(out_file, fourcc, 30, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
    print("Output file written - ", out_file)

    # Loop through the video frames and extract frames with facial features
    frame_number = 0
    frame_ids = []

    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            # Get the tracking data for the current frame
            frame_data = tracking_data.loc[frame_number]

            # Check if the desired facial features are present in the frame
            feature_present = False
            check = 0
            for feature_name in feature_names:
                if feature_name in frame_data.loc[x] and frame_data.loc[x].loc[feature_name].loc['likelihood'] > confidence_threshold:
                    check += 1
            if check==4:
                feature_present = True

            # If the desired facial features are present, save the frame to the output video
            if feature_present: #check==4
                cv2.imshow('output', frame)
                out.write(frame)
                frame_ids.append(frame_number)
            # Display the output
            #cv2.imshow('output', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            print(round(frame_number/tot_frame*100, 2), end = '\r')
            # Increment the frame number
            frame_number += 1
        else:
            break
    print("Done")
    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    return frame_ids    

In [12]:
os.chdir('/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids')

In [13]:
os.getcwd()

'/home/yramakrishna/DeepLabCut/conda-environments/Codes/All_vids'

In [14]:
if not os.path.exists('Cropped_vids'):
        os.makedirs('Cropped_vids')
os.chdir('Cropped_vids')

In [15]:
crop_source = []
if len(h5files)==len(vid_to_crop):
    for i in h5files:
        found = ''
        for j in vid_to_crop:
            x = os.path.splitext(i)[0].rstrip('_filtered.h5')
            y = os.path.splitext(j)[0].rstrip('filtered_labeled.mp4')
            if x==y:
                found = j
        k = os.path.basename(i)
        l = 'DLC' + k.split('DLC')[1] + 'DLC' + k.split('DLC')[2].rstrip('_filtered.h5')
        # print(l)
        # print(i)
        # print(found)
        # print(k)
        # print(l)
        # print('')
        crop_source.append([i,found, l])
c_s = np.array(crop_source)


In [16]:
# # crop_it(c_s[0])
# for i in c_s:
#     crop_it(i)

In [17]:
all_frames = []
# crop_it(c_s[0])
for i in c_s:
    x = crop_it(i)
    all_frames.append([i[1],x])

Starting Cropping
Output file written -  AAACrpoutput_videovin_20210528_1002_XBI19_reversal_learning_foraging_SC3_DCS_frontCameradownsampled_GCDLC_resnet50_DLCApr28shuffle1_1030000_filtered_labeled.mp4


QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread

Done0
Starting Cropping
Output file written -  AAACrpoutput_videovin_20210604_0907_XBI19_reversal_learning_foraging_SC3_DCS_front_cameradownsampled_GCDLC_resnet50_DLCApr28shuffle1_1030000_filtered_labeled.mp4


QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread

Done0
Starting Cropping
Output file written -  AAACrpoutput_videovin_20210526_1022_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled_GCDLC_resnet50_DLCApr28shuffle1_1030000_filtered_labeled.mp4


...

Done0
Starting Cropping
Output file written -  AAACrpoutput_videoken_20210507_1032_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled_GCDLC_resnet50_DLCApr28shuffle1_1030000_filtered_labeled.mp4


QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread

Done0
Starting Cropping
Output file written -  AAACrpoutput_videoken_20210504_1236_XBI19_reversal_learning_foraging_SC3_DCS_frontcameradownsampled_GCDLC_resnet50_DLCApr28shuffle1_1030000_filtered_labeled.mp4


QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread (0xc2934f0).
Cannot move to target thread (0xbce6d00)

QObject::moveToThread: Current thread (0xbce6d00) is not the object's thread

Done0
